# Face Recogniction using Keras-OpenFace

Below the is the implementation of a very basic face recognizer which can identify the face of the person showing on a web cam.
The implementation is inspired by two path breaking papers on facial recognition using deep convoluted neural network, namely FaceNet and DeepFace.

I have used pre trained model Keras-OpenFace which is an open source Keras implementation of the OpenFace (Originally Torch implemented) 

The pretrained model that I have used is by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.

# Import the prerequisite libraries

We will be importing utils.py from https://github.com/iwantooxxoox/Keras-OpenFace/blob/master/utils.py (available with code) which contains utility functions to create the neural network and load the weights assoiated with it.

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Building the neural network model
The model built here is based on FaceNet's Inception model.

The implementation of model is available at: https://github.com/iwantooxxoox/Keras-OpenFace

### Model Layers

In [4]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

Instructions for updating:
Colocations handled automatically by placer.


# Loading the model with pretrained weights

FaceNet is trained by minimizing the triplet loss. I will be  loading a previously trained model. weights are avaiable at https://github.com/iwantooxxoox/Keras-OpenFace in the "weights" folder which is also provided in this source.

This can take a couple of minutes to execute and depends on the speed of your machine.


In [5]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

## The image_to_embedding function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. **"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [6]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## The recognize_face function
This function calculates similarity between the captured image and the images that are already been stored. It passes the image to the trained neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating L2 Euclidean distance. 

If the minimum L2 distance between two embeddings is less than a threshpld (here I have taken the threashhold as 0.75 (which can be adjusted) then we have a match.

In [7]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.75:
        return str(name)
    else:
        return None

##  The create_input_image_embeddings function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by feed forwarding the images to a trained neural network. It creates a dictionary with key as the name of the face and value as embedding


## The recognize_faces_in_cam function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [8]:
import glob

def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        a = np.double(img) + 30
        img = np.uint8(a)
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            

            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
Following code captures 10 face images of the person. They all are stored in **"images"** folder with the name Face_1 to Face_10. Select a good captured image from the set of 10 images. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person using one shot learning.

In [2]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("images/Face_" + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 30 face sample and stop video
         break
cam.release()
cv2.destroyAllWindows()

In [24]:
# img = cv2.imread('pro_pic.jpg')
# faces = face_detector.detectMultiScale(img, 1.3, 5)
# for (x,y,w,h) in faces:
#     x1 = x
#     y1 = y
#     x2 = x+w
#     y2 = y+h
#     cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
#     count += 1
#     # Save the captured image into the datasets folder
#     cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])

# Running the face recognizer :-)

In [33]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)

Euclidean distance from Abhishek is 1.1273216
Euclidean distance from Abhishek1 is 1.0204419
Euclidean distance from Abhishek is 0.88290167
Euclidean distance from Abhishek1 is 0.87122107
Euclidean distance from Abhishek is 1.0443152
Euclidean distance from Abhishek1 is 0.91746503
Euclidean distance from Abhishek is 1.1951636
Euclidean distance from Abhishek1 is 1.0256407
Euclidean distance from Abhishek is 0.9706499
Euclidean distance from Abhishek1 is 0.76138884
Euclidean distance from Abhishek is 0.9734893
Euclidean distance from Abhishek1 is 0.8305181
Euclidean distance from Abhishek is 1.0408598
Euclidean distance from Abhishek1 is 0.8746132
Euclidean distance from Abhishek is 0.9248526
Euclidean distance from Abhishek1 is 0.8019335
Euclidean distance from Abhishek is 0.88917744
Euclidean distance from Abhishek1 is 0.986266
Euclidean distance from Abhishek is 1.0215684
Euclidean distance from Abhishek1 is 0.7209209
Euclidean distance from Abhishek is 0.93706316
Euclidean distance 

Euclidean distance from Abhishek is 1.1575611
Euclidean distance from Abhishek1 is 0.79673266
Euclidean distance from Abhishek is 0.6846579
Euclidean distance from Abhishek1 is 0.55037475
Euclidean distance from Abhishek is 0.81635076
Euclidean distance from Abhishek1 is 0.5301591
Euclidean distance from Abhishek is 0.882832
Euclidean distance from Abhishek1 is 0.62914157
Euclidean distance from Abhishek is 1.0940272
Euclidean distance from Abhishek1 is 0.7587464
Euclidean distance from Abhishek is 0.8113332
Euclidean distance from Abhishek1 is 0.51277655
Euclidean distance from Abhishek is 0.9425007
Euclidean distance from Abhishek1 is 0.6390808
Euclidean distance from Abhishek is 1.0664047
Euclidean distance from Abhishek1 is 0.76848626
Euclidean distance from Abhishek is 0.76968235
Euclidean distance from Abhishek1 is 0.48648286
Euclidean distance from Abhishek is 0.5971432
Euclidean distance from Abhishek1 is 0.52900654
Euclidean distance from Abhishek is 0.5744122
Euclidean distan

Euclidean distance from Abhishek is 0.8283621
Euclidean distance from Abhishek1 is 0.79391056
Euclidean distance from Abhishek is 0.6633204
Euclidean distance from Abhishek1 is 0.888382
Euclidean distance from Abhishek is 0.93032753
Euclidean distance from Abhishek1 is 0.88479316
Euclidean distance from Abhishek is 0.6574397
Euclidean distance from Abhishek1 is 0.80733836
Euclidean distance from Abhishek is 0.74937034
Euclidean distance from Abhishek1 is 0.83027023
Euclidean distance from Abhishek is 0.71024626
Euclidean distance from Abhishek1 is 0.74604696
Euclidean distance from Abhishek is 1.2536502
Euclidean distance from Abhishek1 is 1.3750829
Euclidean distance from Abhishek is 0.7180119
Euclidean distance from Abhishek1 is 0.90556586
Euclidean distance from Abhishek is 0.70649874
Euclidean distance from Abhishek1 is 1.0397742
Euclidean distance from Abhishek is 0.67770207
Euclidean distance from Abhishek1 is 0.88957274
Euclidean distance from Abhishek is 1.1888381
Euclidean dis

Euclidean distance from Abhishek is 1.0213573
Euclidean distance from Abhishek1 is 0.80512583
Euclidean distance from Abhishek is 0.6982958
Euclidean distance from Abhishek1 is 0.76140845
Euclidean distance from Abhishek is 1.0155853
Euclidean distance from Abhishek1 is 0.88906896
Euclidean distance from Abhishek is 1.0703781
Euclidean distance from Abhishek1 is 0.8851587
Euclidean distance from Abhishek is 1.050696
Euclidean distance from Abhishek1 is 0.82328063
Euclidean distance from Abhishek is 0.91158795
Euclidean distance from Abhishek1 is 0.88880676
Euclidean distance from Abhishek is 0.92749375
Euclidean distance from Abhishek1 is 0.7999923
Euclidean distance from Abhishek is 0.9194626
Euclidean distance from Abhishek1 is 1.1962851
Euclidean distance from Abhishek is 1.0066711
Euclidean distance from Abhishek1 is 0.75229454
Euclidean distance from Abhishek is 1.0669703
Euclidean distance from Abhishek1 is 0.843245
Euclidean distance from Abhishek is 1.1299236
Euclidean distance

Euclidean distance from Abhishek is 0.5849713
Euclidean distance from Abhishek1 is 0.8376434
Euclidean distance from Abhishek is 0.68167144
Euclidean distance from Abhishek1 is 0.90313405
Euclidean distance from Abhishek is 0.7104659
Euclidean distance from Abhishek1 is 0.9523089
Euclidean distance from Abhishek is 0.62597173
Euclidean distance from Abhishek1 is 0.8267412
Euclidean distance from Abhishek is 0.58594066
Euclidean distance from Abhishek1 is 0.9454785
Euclidean distance from Abhishek is 0.67986035
Euclidean distance from Abhishek1 is 0.82367975
Euclidean distance from Abhishek is 0.6590367
Euclidean distance from Abhishek1 is 0.80229247
Euclidean distance from Abhishek is 0.66226786
Euclidean distance from Abhishek1 is 0.8545892
Euclidean distance from Abhishek is 0.5812983
Euclidean distance from Abhishek1 is 0.8873795
Euclidean distance from Abhishek is 0.58767825
Euclidean distance from Abhishek1 is 0.92117834
Euclidean distance from Abhishek is 0.65317136
Euclidean dis

# A second approach for recognizing faces

In this approach we have/create a dataset of faces of the individuals we want to recognize.<br>
We feed these images to our pre-trained model to obtain 128 length vector embeddings corresponding to each of the sample faces. Using these embeddings as our dataset we train a classifier which is then used to recognize different faces.
A drawback of this approach is that it will always classify into one of the classes. Though this can be easily mitigated using a confidence score or other such criterion.

In [9]:
persons = {0:'Abhishek',
         1:'Swati',
         2:'Person_1',
         3:'Person_2'}

#### List of directries containing the face images of different individuals

In [10]:
aks_dir = '/home/aks/Desktop/Profile_pics/aks/aks'
swati_dir = '/home/aks/Desktop/Profile_pics/aks/swati'
#other_dir = '/home/aks/Desktop/Profile_pics/aks/others'

dir_list = [aks_dir, swati_dir]
embeddings = []
y = []

#### Creating Image embeddings for training the classifier

In [11]:
import os

lbl = 0
for directory in dir_list:
    for f in os.listdir(directory):
        file = directory+'/'+f
        img = cv2.imread(file)
        embedding = image_to_embedding(img, model)
        embeddings.append(embedding)
        y.append(lbl)
    lbl = lbl + 1

print(len(embeddings), len(y))
x = np.array(embeddings)
x = x.reshape(x.shape[0],128)
print(x.shape)

24 24
(24, 128)


#### Commented Code

In [12]:
# import os

# faces_aks = os.listdir(aks_dir)
# faces_others = os.listdir(other_dir)
# faces_swati = os.listdir(swati_dir)

# aks_embeddings = []
# other_embeddings = []
# swati_embeddings = []

# print(len(faces_swati))

In [13]:
# for f in faces_aks:
#     file = aks_dir+'/'+f
#     img = cv2.imread(file)
#     embedding = image_to_embedding(img, model)
#     aks_embeddings.append(embedding)
# print(len(aks_embeddings))

In [14]:
# for f in faces_others:
#     file = other_dir+'/'+f
#     img = cv2.imread(file)
#     embedding = image_to_embedding(img, model)
#     other_embeddings.append(embedding)

# for f in faces_swati:
#     file = swati_dir+'/'+f
#     img = cv2.imread(file)
#     embedding = image_to_embedding(img, model)
#     swati_embeddings.append(embedding)
# print(len(swati_embeddings))

In [15]:
# y = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2]
# x = aks_embeddings + swati_embeddings
# x = np.array(x)
# x = x.reshape(24,128)

In [16]:
# import pandas as pd
# df = pd.DataFrame(x)

#### Defining and training the classifier

In [17]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN

svc = SVC(gamma='auto')
rfc = RandomForestClassifier(n_estimators=50, random_state=1)
knn = KNN(n_neighbors=5)
adb = AdaBoostClassifier(n_estimators=100, random_state=1)

clf = VotingClassifier(estimators=[
    ('svc',svc),('rfc',rfc),('knn',knn)], voting='hard'
)
clf = knn
clf.fit(x,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

#### recognize_faces_in_cam function

In [18]:
def recognize_faces_in_cam2():
    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        a = np.double(img) + 30
        img = np.uint8(a)
        height, width, channels = frame.shape

        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        print('Faces Detected: ',len(faces))

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

           
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            #identity = recognize_face(face_image, input_embeddings, model)
            embedding = image_to_embedding(face_image, model)
            identity = clf.predict(embedding)
            print(identity)
            
            per = persons.get(int(identity))
            img = cv2.rectangle(frame,(x1, y1),(x2, y2),(50,205,50),2)
            cv2.putText(img, str(per), (x1+5,y1-5), font, 1, (50,205,50), 2)
            
#             if identity == 0:
#                 img = cv2.rectangle(frame,(x1, y1),(x2, y2),(50,205,50),2)
#                 cv2.putText(img, str('Abhishek'), (x1+5,y1-5), font, 1, (50,205,50), 2)
#             elif identity == 2:
#                 img = cv2.rectangle(frame,(x1, y1),(x2, y2),(50,205,50),2)
#                 cv2.putText(img, str('Swati'), (x1+5,y1-5), font, 1, (50,205,50), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

#### recognize_faces_in_img function

In [32]:
def recognize_faces_in_img(filepath):
    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    img = cv2.imread(filepath)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    print('Faces Detected: ',len(faces))
    frame = img
    height, width, channels = frame.shape
    # Loop through all the faces detected 
    identities = []
    for (x, y, w, h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h

           
            
        face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
        #identity = recognize_face(face_image, input_embeddings, model)
        embedding = image_to_embedding(face_image, model)
        identity = clf.predict(embedding)
        print(identity)
        

        if identity == 0:
            img = cv2.rectangle(frame,(x1, y1),(x2, y2),(50,205,50),2)
            cv2.putText(img, str('Abhishek'), (x1+5,y1-5), font, 1, (50,205,50), 2)
        elif identity == 2:
            img = cv2.rectangle(frame,(x1, y1),(x2, y2),(50,205,50),2)
            cv2.putText(img, str('Other'), (x1+5,y1-5), font, 1, (50,205,50), 2)
        
    key = cv2.waitKey(100)
    cv2.imshow("Face Recognizer", img)

    if key == 27: # exit on ESC
        vc.release()
        cv2.destroyAllWindows()
        
    

#### Recognizing faces

##### In cam

In [19]:
try:
    recognize_faces_in_cam2()
except Exception as e:
    print(e)
    cv2.destroyAllWindows()

Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  2
[0]
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces D

Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[1]
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  0
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[1]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[1]
Faces Detected:  1
[1]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[0]
Faces Detected:  1
[1]
Faces D

##### In Image

In [37]:
try:
    #recognize_faces_in_img('/home/aks/Desktop/Profile_pics/test.JPG')
    cv2.imshow('helo',cv2.imread('/home/aks/Desktop/Profile_pics/test.JPG'))
except:
    print('some error')